In [43]:
def recommend_airbnbs(filepath="../ML-exam/data/clustered_airbnb.csv"):
    import pandas as pd

    # Load data
    df = pd.read_csv(filepath)

    # --- Available cities ---
    cities = sorted(df['City'].unique())
    print("Available cities:")
    for c in cities:
        print("-", c)

    # --- User Inputs ---
    print("\n--- Enter your preferences ---")
    try:
        user_budget = float(input("Total budget (€): "))
        duration_input = input("Number of nights (leave blank to maximize duration): ").strip()
        min_bedrooms = int(input("Minimum number of bedrooms: "))
        max_city_dist = float(input("Max distance to city center (e.g. 3.0): "))
        max_metro_dist = float(input("Max distance to metro (e.g. 1.0): "))
        weekend = input("Is your stay during a weekend? (yes/no): ").lower() == "yes"
        city_input = input("Pick a city (leave blank to search all): ").strip()
    except ValueError:
        print("Invalid input. Please try again.")
        return

    # Determine mode
    if duration_input == "":
        mode = "duration"
        user_duration = None
    else:
        mode = "value"
        try:
            user_duration = int(duration_input)
        except ValueError:
            print("Invalid number of nights. Please enter an integer.")
            return

    # --- Filter listings ---
    filtered_df = df.copy()
    if weekend:
        filtered_df = filtered_df[filtered_df['Is_weekend_bool'] == 1]
    if city_input:
        filtered_df = filtered_df[filtered_df['City'].str.lower() == city_input.lower()]
    filtered_df = filtered_df[
        (filtered_df['bedrooms'] >= min_bedrooms) &
        (filtered_df['dist'] <= max_city_dist) &
        (filtered_df['metro_dist'] <= max_metro_dist)
    ].copy()

    if filtered_df.empty:
        print("\nNo listings match your criteria.")
        return

    # --- Compute recommendations ---
    if mode == "duration":
        filtered_df['max_nights'] = (user_budget / filtered_df['realSum']).apply(int)
        recommended = filtered_df.sort_values(by='max_nights', ascending=False)
        display_cols = ['City', 'realSum', 'bedrooms', 'dist', 'metro_dist',
                        'guest_satisfaction_overall', 'max_nights']
    else:
        # Remove listings that exceed total budget
        filtered_df = filtered_df[filtered_df['realSum'] <= user_budget]
        if filtered_df.empty:
            print("\nNo listings within your budget for the selected duration.")
            return
        filtered_df['price_per_day'] = filtered_df['realSum'] / user_duration
        filtered_df['value_score'] = filtered_df['guest_satisfaction_overall'] / filtered_df['price_per_day']
        recommended = filtered_df.sort_values(by='value_score', ascending=False)
        display_cols = ['City', 'realSum', 'bedrooms', 'dist', 'metro_dist',
                        'guest_satisfaction_overall', 'value_score']

    # --- Output Results ---
    print(f"\nTop 10 Recommended Listings (Mode: {mode}):\n")
    print(recommended[display_cols].head(10).to_string(index=False))

In [45]:
recommend_airbnbs()

Available cities:
- Amsterdam
- Athens
- Barcelona
- Berlin
- Budapest
- Lisbon
- London
- Paris
- Rome
- Vienna

--- Enter your preferences ---


Total budget (€):  500
Number of nights (leave blank to maximize duration):  5
Minimum number of bedrooms:  2
Max distance to city center (e.g. 3.0):  2
Max distance to metro (e.g. 1.0):  1
Is your stay during a weekend? (yes/no):  yes
Pick a city (leave blank to search all):  



Top 10 Recommended Listings (Mode: value):

  City  realSum  bedrooms  dist  metro_dist  guest_satisfaction_overall  value_score
Athens    72.41         2   1.7         0.1                          97     6.697970
Athens    69.60         2   1.8         0.4                          92     6.609195
Athens    74.05         3   2.0         0.1                          96     6.482107
Athens    76.63         2   1.8         0.4                          99     6.459611
Athens    81.08         2   1.5         0.4                         100     6.166749
Athens    78.74         2   1.7         0.8                          95     6.032512
Athens    78.74         2   2.0         0.3                          95     6.032512
Athens    78.50         2   1.9         0.9                          94     5.987261
Athens    81.08         2   1.8         0.1                          96     5.920079
Athens    83.43         2   1.4         0.5                          98     5.873187


### How the Value Score Is Calculated

When the user provides a fixed number of nights (i.e., not leaving the "number of nights" field blank), the system calculates a **value score** for each listing to rank the most cost-effective options.

The formula used is:

```python
value_score = guest_satisfaction_overall / price_per_day
```

Where:

* `guest_satisfaction_overall`: A rating from 0 to 100 indicating how satisfied previous guests were.
* `price_per_day`: Calculated as:

  ```python
  price_per_day = realSum / number_of_nights
  ```

  `realSum` is the total cost of the listing (for the entire stay).

### Interpretation

* A **higher value score** indicates **better guest satisfaction per euro spent per day**.
* This helps highlight listings that are not just cheap, but also highly rated — ensuring you get the **best bang for your buck**.